In [1]:
{-# LANGUAGE FlexibleContexts #-}

In [2]:
:load src/stats.hs

In [3]:
import Text.CSV
import Stats
import System.Random
import Data.Maybe
import Data.List
import Data.Ord
import Database.HDBC
import Database.HDBC.Sqlite3
import Graphics.EasyPlot

In [4]:
goog <- parseCSVFromFile "./data/GOOG.csv"
ford <- parseCSVFromFile "./data/FORD.csv"
amzn <- parseCSVFromFile "./data/AMZN.csv"

In [5]:
length $ bbEither goog

3944

In [6]:
movingSum :: [Double] -> [Double] -> [Double]
movingSum xs [] = [sum xs]
movingSum (x:xs) (y:ys) = (sum (x:xs)):movingSum (xs++[y]) ys

In [7]:
movingSum [1,2,3] [4,5,6,7,8,9,10]

[6.0,9.0,12.0,15.0,18.0,21.0,24.0,27.0]

In [8]:
movingAverage :: [Double] -> Int -> [Double]
movingAverage xs windowSize
    | windowSize < 1 = error "Non positive window size"
    | windowSize > length xs = []
    | otherwise = map (/ fromIntegral windowSize) movingSums
    where
        movingSums = movingSum (take windowSize xs) (drop windowSize xs)

In [9]:
movingAverage [1..10] 3

[2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0]

In [10]:
open :: [Double]
open = readIndex goog 2

In [11]:
Just m = medie open

In [12]:
show m

"510.3194009518249"

In [13]:
take 10 open

[51.835709,54.336334,56.528118,55.591629,53.798351,53.773445,54.107193,52.548038,51.661362,51.292744]

In [14]:
length $ movingAverage open 200

3745

In [15]:
rescale :: [Double] -> [Double]
rescale xs = map (\x -> (x-lead)/(diffxs)) xs
    where
        lead = last xs
        maxxs = maximum xs
        minxs = minimum xs
        diffxs = maxxs - minxs

In [16]:
rescale $ take 5 open

[-0.4182589369340985,0.11464963944958854,0.5817410630659015,0.38216574897882977,0.0]

In [17]:
gen <- newStdGen

In [18]:
values = take 100000 (randomRs (1,100) gen) :: [Double]

In [19]:
:type randomRs

randomRs :: forall g a. (RandomGen g, Random a) => (a, a) -> g -> [a]

In [21]:
chunk [] = []
chunk list = take 100 list : chunk (drop 100 list)

In [22]:
chuncked = chunk values

In [23]:
length chuncked

1000

In [24]:
medii = map medie chuncked

In [25]:
take 10 medii

[Just 50.97357804165463,Just 53.691870627371046,Just 49.252107435383635,Just 52.19077235477594,Just 49.98467949314086,Just 50.07387036557679,Just 47.45111253387588,Just 43.99936131133777,Just 50.47007529123091,Just 48.39067039448465]

In [27]:
pairs = runLengthEncoding $ sort (map (\(Just m) -> round m) medii)

In [28]:
take 5 pairs

[(41,2),(42,1),(43,7),(44,12),(45,23)]